## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Общая информация 

In [1]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Рассмотрим полученную информацию подробнее.

Всего в таблице 12 столбцов:
- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

7 столбцов содержат качественные переменные: *education, education_id, family_status, family_status_id, gender, income_type, purpose*; 4 столбца - количественные переменые: *children, days_employed, dob_years, total_income*; и 1 столбец - логические переменные: *debt*.

В столбце *days_employed* - дробные числа, заменим их на целые числа. В этом же столбце имеются отрицательные значения, которые необходимо заменить на положительные. Типы данных остальных столбцов соответствуют их содержимому. 
В столбце *education* строки отличаются регистром букв: для среднего образования есть значения - среднее, Среднее, и СРЕДНЕЕ.
В столбце *purpose* цели не структурированы, одни и те же цели имеют разные формы: очевидно, что "сыграть свадьбу" и "на проведение свадьбы" - это одна и та же цель.
Кроме того, в следующих столбцах есть пропуски: *days_employed* и *total_income*. 

### Вывод

Таблица содержит 21525 строк - данных о платежеспособностях клиентов банка. При этом персональные данные клиентов отсутствуют. Перед изучением влияния характеристик клиента, влияющих на возврат кредита в срок, необходимо: 
* обработать пропуски, 
* изменить тип данных в столбце *days_employed*,
* провести поиск дубликатов,
* выполнить поиск дубликатов с учётом регистра в столбце *education*,
* провести категоризацию данных по целям получения кредита, использую лемматизацию.

### Шаг 2. Предобработка данных

### Обработка пропусков

Судя по общей информации из таблицы, количество пропусков в столбцах *days_employed* и *total_income* совпадает. Возможно, это не случайно, и в таблице есть строки, с пропущенными значениями одновременно в столбцах *days_employed* и *total_income*. Посчитаем количество пропусков в столбце *days_employed*.

In [3]:
df[df['days_employed'].isnull()].count()

children            2174
days_employed          0
dob_years           2174
education           2174
education_id        2174
family_status       2174
family_status_id    2174
gender              2174
income_type         2174
debt                2174
total_income           0
purpose             2174
dtype: int64

Проверим, что количество строк, в которых одновременно есть пропуски в столбцах *days_employed* и *total_income*, равно 2174. 

In [4]:
null_in_rows = df[(df['days_employed'].isnull()) & (df['total_income'].isnull())]
null_in_rows.count()

children            2174
days_employed          0
dob_years           2174
education           2174
education_id        2174
family_status       2174
family_status_id    2174
gender              2174
income_type         2174
debt                2174
total_income           0
purpose             2174
dtype: int64

Т.о. пропуски в столбцах находятся в одних и тех же строках. Это могут быть клиенты банка, которые брали кредиты на небольшие суммы без подтверждения трудового стажа и ежемесячного дохода. Т.к. 2174 строки составляют значительную часть данных - 10%, а также в этих строках есть информация, необходимая для анализа (столбцы *children*, *family_status(family_status_id)* и *debt* заполнены), то не будем удалять строки с пропущенными значениями, а заменим пропуски на медианные значения, отфильтрованные по типу занятости.

Выясним, какие типы занятости есть в строках с пропущенными значениями.

In [5]:
null_in_rows['income_type'].value_counts()

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

Перед заполнением пропущенных данных избавимся от отрицательных значений в столбце *days_employed*.

#### Обработка отрицательных значений days_employed 

Изучим отрицательные значения подробнее. Рассмотрим, есть ли зависимость от типа занятости.

In [6]:
df[df['days_employed'] < 0].groupby('income_type')['income_type'].count()

income_type
в декрете              1
госслужащий         1312
компаньон           4577
предприниматель        1
сотрудник          10014
студент                1
Name: income_type, dtype: int64

In [7]:
df[df['days_employed'] > 0].groupby('income_type')['income_type'].count()

income_type
безработный       2
пенсионер      3443
Name: income_type, dtype: int64

Т.о. все отрицательные значения относятся только к типам занятости - в декрете, госслужащий, компаньон, предприниматель, сотрудник, студент. 

Проверим, что по модулю отрицательные значения верны. Для этого сгруппируем их по возрасту и сравним средний трудовой стаж. Для составления возрастных групп используем квартили.

In [8]:
df['dob_years'].describe()

count    21525.000000
mean        43.293380
std         12.574584
min          0.000000
25%         33.000000
50%         42.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64

Следовательно, в таблице есть возраст = 0. 

In [9]:
df[df['dob_years'] == 0].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,346541.618895,0,Среднее,1,женат / замужем,0,F,пенсионер,0,71291.522491,автомобиль
149,0,-2664.273168,0,среднее,1,в разводе,3,F,сотрудник,0,70176.435951,операции с жильем
270,3,-1872.663186,0,среднее,1,женат / замужем,0,F,сотрудник,0,102166.458894,ремонт жилью
578,0,397856.565013,0,среднее,1,женат / замужем,0,F,пенсионер,0,97620.687042,строительство собственной недвижимости
1040,0,-1158.029561,0,высшее,0,в разводе,3,F,компаньон,0,303994.134987,свой автомобиль


Т.к. в текущем проекте мы не исследуем, как влияет возраст на возврат кредита в срок, и в строках с нулевыми значениями есть необходимые для изучения данные, то не будем их изменять, но и не будем учитывать в сравнении возрастных групп.

In [10]:
days_employed_negative = df[df['days_employed'] < 0]
junior_negative_mean = days_employed_negative[(days_employed_negative['dob_years'] > 0) & (days_employed_negative['dob_years'] < 33)]['days_employed'].mean()
adult_negative_mean = days_employed_negative[(days_employed_negative['dob_years'] >= 33) & (days_employed_negative['dob_years'] < 42)]['days_employed'].mean()
age_negative_mean = days_employed_negative[(days_employed_negative['dob_years'] >= 42) & (days_employed_negative['dob_years'] < 53)]['days_employed'].mean()
elderly_negative_mean = days_employed_negative[days_employed_negative['dob_years'] >= 53]['days_employed'].mean()

if abs(elderly_negative_mean) > abs(age_negative_mean) > abs(adult_negative_mean) > abs(junior_negative_mean):
    print('Средний трудовой стаж соответствует возрастным группам')
else:
    print('Средний трудовой стаж не соответствует возрастным группам') 

Средний трудовой стаж соответствует возрастным группам


Можно предположить, что данные о стаже по модулю правильные. Изменим отрицательные значения, на положительные.

In [11]:
df['days_employed'] = abs(df['days_employed'])  
df[df['days_employed'] < 0].count()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

#### Замена пропущенных значений

Вычислим медианные значения для каждого типа занятости по столбцам *days_employed* и *total_income*.

In [12]:
median_days_employed = df.groupby('income_type')['days_employed'].median()
median_days_employed

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

In [13]:
median_total_income = df.groupby('income_type')['total_income'].median()
median_total_income

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64

В *median_days_employed* выделяются два значения: для безработных - 366413.652744, и для пенсионеров - 365213.306266. Т.е. 1003,87 и 1000,58 лет соответственно. Исследуем подробнее, как появились эти неправдоподобные данные.

Посчитаем, у скольких безработных некорректный стаж. Для этого рассмотрим, у скольких безработных стаж больше 100 лет = 36500 дней. Предположительно, у всех безработных трудовой стаж должен быть меньше 100 лет.

In [14]:
de_max = 36500
unemployed_total = df[df['income_type'] == 'безработный']['days_employed'].count()
unemployed_over_de_max = df[(df['income_type'] == 'безработный') & (df['days_employed'] > de_max)]['days_employed'].count()

if unemployed_total == unemployed_over_de_max:
    print('У всех {} безработных некорректный стаж.'.format(unemployed_total))
else:
    print('В {} безработных из {} некорректный стаж.'.format(unemployed_over_de_max, unemployed_total))  

У всех 2 безработных некорректный стаж.


Т.к. нет данных по среднему трудовому стажу у безработных, то мы не можем скорректировать данные и, сответственно, использовать их в анализе. Из-за схожести стажа безработных со стажем пенсионеров можно предположить, что некорректно введен тип занятости для этих строк - должен быть "пенсионер". В данном случае 2 строки с безработными удалим, т.к. в них не содержатся важные данные для ответа на поставленные вопросы.

In [15]:
df = df[df['income_type'] != 'безработный']
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21523 entries, 0 to 21524
Data columns (total 12 columns):
children            21523 non-null int64
days_employed       19349 non-null float64
dob_years           21523 non-null int64
education           21523 non-null object
education_id        21523 non-null int64
family_status       21523 non-null object
family_status_id    21523 non-null int64
gender              21523 non-null object
income_type         21523 non-null object
debt                21523 non-null int64
total_income        19349 non-null float64
purpose             21523 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


Рассмотрим пенсионеров. У пенсионеров также слишком большой стаж. Подсчитаем, сколько пенсионеров имеют трудовой стаж больше 36500 дней. Ожидание - у всех пенсионеров трудовой стаж должен быть меньше 100 лет.

In [16]:
pensioner_over_de_max = df[(df['income_type'] == 'пенсионер') & (df['days_employed'] > de_max)]['days_employed'].count()
pensioner_total = df[df['income_type'] == 'пенсионер']['days_employed'].count() 

if pensioner_over_de_max == pensioner_total:
    print('У всех {} пенсионеров некорректный стаж.'.format(pensioner_over_de_max))
else:
    print('В {} пенсионерах из {} некорректный стаж.'.format(pensioner_over_de_max, pensioner_total))

У всех 3443 пенсионеров некорректный стаж.


Рассмотрим, насколько средний трудовой стаж из таблицы больше среднего трудового стажа при выходе на пенсию в России.

Средний трудовой стаж в РФ при выходе на пенсию составляет 34,5 года, это 12592,5 дня (https://ria.ru/20131119/977986173.html, данные за 2013 год). Т.к. пенсионная реформа начата только в 2019 году, то допустим, что средний трудовой стаж в России на момент формирования таблицы изменился незначительно с 2013 года.

In [17]:
real_mean_pensioner_de = 12592.5
pensioner_mean = df[df['income_type'] == 'пенсионер']['days_employed'].mean()
index = pensioner_mean / real_mean_pensioner_de
index

28.98578449433085

Мы получили *index* - коэффициент, указывающий во сколько раз средний трудовой стаж пенсионеров из таблицы больше среднего трудового стажа пенсионеров в России. Возможно при выгрузке данных произошла ошибка, данные по стажу умножили на среднее количество дней в месяце. Чтобы получить приближенные к реальным данные трудового стажа пенсионеров, поделим их на *index*.

In [18]:
mask = (df['income_type'] == 'пенсионер') & (df['days_employed'] > 0)
df.loc[mask, 'days_employed'] = df.loc[mask, 'days_employed'] / index   
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,11739.067201,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Заполним пропуски в столбце *days_employed*. Для этого выделим список уникальных типов занятости и в цикле заполним пропуски для текущего типа занятости медианой.

In [19]:
median_de = df.groupby('income_type')['days_employed'].median()
median_de

income_type
в декрете           3296.759962
госслужащий         2689.368353
компаньон           1547.382223
пенсионер          12599.738549
предприниматель      520.848083
сотрудник           1574.202821
студент              578.751554
Name: days_employed, dtype: float64

In [1]:
median_de_index = median_de.index.tolist()

NameError: name 'median_de' is not defined

In [21]:
for income_type in median_de_index:
    mask = (df['days_employed'].isnull()) & (df['income_type'] == income_type)
    df.loc[mask, 'days_employed'] = median_de[income_type]

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21523 entries, 0 to 21524
Data columns (total 12 columns):
children            21523 non-null int64
days_employed       21523 non-null float64
dob_years           21523 non-null int64
education           21523 non-null object
education_id        21523 non-null int64
family_status       21523 non-null object
family_status_id    21523 non-null int64
gender              21523 non-null object
income_type         21523 non-null object
debt                21523 non-null int64
total_income        19349 non-null float64
purpose             21523 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


Аналогично, заполним пропуски в столбце *total_income*.

In [23]:
median_ti = df.groupby('income_type')['total_income'].median()
median_ti

income_type
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64

In [24]:
median_ti_index = median_ti.index.tolist()

In [25]:
for income_type in median_ti_index:
    mask = (df['total_income'].isnull()) & (df['income_type'] == income_type)
    df.loc[mask, 'total_income'] = median_ti[income_type]

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21523 entries, 0 to 21524
Data columns (total 12 columns):
children            21523 non-null int64
days_employed       21523 non-null float64
dob_years           21523 non-null int64
education           21523 non-null object
education_id        21523 non-null int64
family_status       21523 non-null object
family_status_id    21523 non-null int64
gender              21523 non-null object
income_type         21523 non-null object
debt                21523 non-null int64
total_income        21523 non-null float64
purpose             21523 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


### Вывод

* Обработаны пропущенные значения в столбцах *days_employed* и *total_income*: пропуски заменены на медианы, рассчитанные по каждому типу занятости.
* Отрицательные значения в столбце *days_employed* заменены на модули.
* Скорректирован трудовой стаж пенсионеров: значения поделены на вычисленный коэффициент - отношение среднего стажа пенсионеров из таблицы к среднему стажу пенсионеров в России.

### Замена типа данных

Для удобства  расчетов изменим тип данных в столбцах *days_employed* и *total_income* c вещественного на целочисленный. Воспользуемся методом astype(), который переводит значения в нужный тип. Проверим полученный результат, вызвав метод info().

In [27]:
df['days_employed'] = df['days_employed'].astype('int')

In [28]:
df['total_income'] = df['total_income'].astype('int')

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21523 entries, 0 to 21524
Data columns (total 12 columns):
children            21523 non-null int64
days_employed       21523 non-null int64
dob_years           21523 non-null int64
education           21523 non-null object
education_id        21523 non-null int64
family_status       21523 non-null object
family_status_id    21523 non-null int64
gender              21523 non-null object
income_type         21523 non-null object
debt                21523 non-null int64
total_income        21523 non-null int64
purpose             21523 non-null object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB


In [30]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,11739,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


### Вывод

Изменен тип данных в столбцах трудовой стаж и ежемесячный доход на целочисленный.

### Обработка дубликатов

Проверим наличие дубликатов в столбцах.

In [31]:
df['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

В столбце *children* есть значения -1 и 20. Рассмотрим их подробнее.

In [32]:
df[df['children'] == 20].count()

children            76
days_employed       76
dob_years           76
education           76
education_id        76
family_status       76
family_status_id    76
gender              76
income_type         76
debt                76
total_income        76
purpose             76
dtype: int64

In [33]:
df[df['children'] == 20].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
606,20,880,21,среднее,1,женат / замужем,0,M,компаньон,0,145334,покупка жилья
720,20,855,44,среднее,1,женат / замужем,0,F,компаньон,0,112998,покупка недвижимости
1074,20,3310,56,среднее,1,женат / замужем,0,F,сотрудник,1,229518,получение образования
2510,20,2714,59,высшее,0,вдовец / вдова,2,F,сотрудник,0,264474,операции с коммерческой недвижимостью
2941,20,2161,0,среднее,1,женат / замужем,0,F,сотрудник,0,199739,на покупку автомобиля


In [34]:
df[df['children'] == -1].count()

children            47
days_employed       47
dob_years           47
education           47
education_id        47
family_status       47
family_status_id    47
gender              47
income_type         47
debt                47
total_income        47
purpose             47
dtype: int64

In [35]:
df[df['children'] == -1].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
291,-1,4417,46,среднее,1,гражданский брак,1,F,сотрудник,0,102816,профильное образование
705,-1,902,50,среднее,1,женат / замужем,0,F,госслужащий,0,137882,приобретение автомобиля
742,-1,3174,57,среднее,1,женат / замужем,0,F,сотрудник,0,64268,дополнительное образование
800,-1,12074,54,среднее,1,Не женат / не замужем,4,F,пенсионер,0,86293,дополнительное образование
941,-1,12599,57,Среднее,1,женат / замужем,0,F,пенсионер,0,118514,на покупку своего автомобиля


У 76 клиентов - по 20 детей, при этом нет ни одного клиента с количеством детей от 6 до 19. Появление значений "20", также как и 47 значений "-1" вызваны человеческим фактором: неверно введены данные. Изменим 20 на 2 и -1 на 1.

In [36]:
mask = (df['children'] == 20)
df.loc[mask, 'children'] = 2

In [37]:
mask = (df['children'] == -1)
df.loc[mask, 'children'] = 1

In [38]:
df['children'].unique()

array([1, 0, 3, 2, 4, 5])

Избавимся от дубликатов в столбце *education*, обнаруженные в Шаге 1. Чтобы учесть дубликаты, все символы в строке приведем к нижнему регистру вызовом метода *lower()*.

In [39]:
df['education'].value_counts()

среднее                13749
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   267
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

In [40]:
df['education'] = df['education'].str.lower()
df['education'].value_counts()

среднее                15232
высшее                  5259
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Проверим наличие дубликатов в столбце *family_status*.

In [41]:
df['family_status'].value_counts()

женат / замужем          12379
гражданский брак          4176
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

Дубликатов в столбце *family_status* не обнаружено. Исследуем столбец *gender*.

In [42]:
df['gender'].value_counts()

F      14235
M       7287
XNA        1
Name: gender, dtype: int64

Значение *XNA* не понятно, необходимо уточнение от предоставившего данные о клиентах, что это за пол. Заменять или удалять значение не будем, т.к. пол в данном проекте не исследуется на влияние на платежеспособность клиента.

In [43]:
df[df['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,2358,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905,покупка недвижимости


При изучении общей информации о таблице (Шаг 1.) также были выявлено наличие дубликатов в столбце *purpose*. Для того, чтобы избавится от них, лемматизируем цели и выделим категории, с которыми связаны цели получения кредита, в следующем разделе.

In [44]:
df['purpose'].value_counts()


свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
операции с жильем                         653
покупка жилья для сдачи                   652
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              611
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

Проверим, есть ли в таблице строки с идентичной информацией.

In [45]:
df.duplicated().sum()

71

Для удаления дубликатов воспользуемся методом *drop_duplicates()*.

In [46]:
df = df.drop_duplicates().reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21452 entries, 0 to 21451
Data columns (total 12 columns):
children            21452 non-null int64
days_employed       21452 non-null int64
dob_years           21452 non-null int64
education           21452 non-null object
education_id        21452 non-null int64
family_status       21452 non-null object
family_status_id    21452 non-null int64
gender              21452 non-null object
income_type         21452 non-null object
debt                21452 non-null int64
total_income        21452 non-null int64
purpose             21452 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Вывод

В таблице удалены одинаковые строки с идентичной информацией. 
Дубликаты выявлены в столбцах *children*, *education* и *purpose*. В столбце *children* заменены неправдоподобные значения -1 и 20 на 1 и 2 соответственно. В столбце *education* избавились от дубликатов, приведя значения к нижнему регистру. Работа с дубликатами в столбце *purpose* продолжается.

### Лемматизация

Лемматизируем значения в столбце *purpose*.

In [47]:
from pymystem3 import Mystem
m = Mystem()

In [48]:
df['lemmas'] = df['purpose'].apply(m.lemmatize)
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]"
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]"
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]"
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]"
4,0,11739,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]"
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,"[покупка, , жилье, \n]"
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,"[операция, , с, , жилье, \n]"
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,"[образование, \n]"
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,"[на, , проведение, , свадьба, \n]"
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,"[покупка, , жилье, , для, , семья, \n]"


Для того, чтобы выделить категории для целей получения кредита, лемматизируем уникальные значения целей и подсчитаем число упоминаний лемматизированных слов.

In [49]:
purpose = df['purpose'].unique()
purpose

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [50]:
lemmas = m.lemmatize(' '.join(purpose))
' '.join(purpose)

'покупка жилья приобретение автомобиля дополнительное образование сыграть свадьбу операции с жильем образование на проведение свадьбы покупка жилья для семьи покупка недвижимости покупка коммерческой недвижимости покупка жилой недвижимости строительство собственной недвижимости недвижимость строительство недвижимости на покупку подержанного автомобиля на покупку своего автомобиля операции с коммерческой недвижимостью строительство жилой недвижимости жилье операции со своей недвижимостью автомобили заняться образованием сделка с подержанным автомобилем получение образования автомобиль свадьба получение дополнительного образования покупка своего жилья операции с недвижимостью получение высшего образования свой автомобиль сделка с автомобилем профильное образование высшее образование покупка жилья для сдачи на покупку автомобиля ремонт жилью заняться высшим образованием'

In [51]:
from collections import Counter
Counter(lemmas)

Counter({'покупка': 10,
         ' ': 96,
         'жилье': 7,
         'приобретение': 1,
         'автомобиль': 9,
         'дополнительный': 2,
         'образование': 9,
         'сыграть': 1,
         'свадьба': 3,
         'операция': 4,
         'с': 5,
         'на': 4,
         'проведение': 1,
         'для': 2,
         'семья': 1,
         'недвижимость': 10,
         'коммерческий': 2,
         'жилой': 2,
         'строительство': 3,
         'собственный': 1,
         'подержать': 2,
         'свой': 4,
         'со': 1,
         'заниматься': 2,
         'сделка': 2,
         'получение': 3,
         'высокий': 3,
         'профильный': 1,
         'сдача': 1,
         'ремонт': 1,
         '\n': 1})

По наибольшему числу упоминаний можно выделить следующие категории: недвижимость, автомобиль, образование, жилье, свадьба. Жилье отнесем к недвижимости. Т.о. получим 4 категории: 
* недвижимость, 
* автомобиль, 
* образование, 
* свадьба.

### Вывод

В новый столбец *lemmas* записали лемматизированные слова из целей получения кредита. Выделили 4 категории целей получения кредита.

### Категоризация данных

#### Категоризация по целям получения кредита

Проведем категоризацию по целям получения кредита.

In [52]:
def get_purpose_group(x):
    if 'свадьба' in x:
        return 'свадьба'
    if 'автомобиль' in x:
        return 'автомобиль'
    if 'образование' in x:
        return 'образование'
    if 'жилье' in x or 'недвижимость' in x:
        return 'недвижимость'
    return 'другое'

In [53]:
df['purpose_group'] = df['lemmas'].apply(get_purpose_group)

In [54]:
df.groupby('purpose_group')['purpose_group'].count()

purpose_group
автомобиль       4306
недвижимость    10809
образование      4013
свадьба          2324
Name: purpose_group, dtype: int64

#### Категоризация по ежемесячному доходу

Проведем категоризацию по ежемесячному доходу. Выделим категории на основе квартилей.

In [55]:
df['total_income'].describe()

count    2.145200e+04
mean     1.653227e+05
std      9.818891e+04
min      2.066700e+04
25%      1.076290e+05
50%      1.425940e+05
75%      1.958188e+05
max      2.265604e+06
Name: total_income, dtype: float64

In [56]:
income_group = [['1', 20666, 107801],
                ['2', 107801, 142594],
               ['3', 142594, 195546],
               ['4', 195546, 2265605]]
columns = ['group_name','income_from','income_to']
income_group_table = pd.DataFrame(data = income_group, columns = columns)
income_group_table

,group_name,income_from,income_to
0,1,20666,107801
1,2,107801,142594
2,3,142594,195546
3,4,195546,2265605


In [57]:
def get_income_group(x):
    if x < 107801:
        return '1'
    if x >= 107801:
        if x < 142594:
            return '2'
        if x < 195546:
            return '3'
    return '4'

Проверим работу функции.

In [58]:
get_income_group(3)

'1'

In [59]:
get_income_group(107801)

'2'

In [60]:
get_income_group(195545)

'3'

In [61]:
get_income_group(195546)

'4'

Функция работает верно. Добавим и заполним столбец с категориями по доходу.

In [62]:
df['total_income_group'] = df['total_income'].apply(get_income_group)

In [63]:
df.groupby('total_income_group')['total_income_group'].count()

total_income_group
1    5381
2    4391
3    6299
4    5381
Name: total_income_group, dtype: int64

#### Категоризация по уровню образования

Проведем категоризацию по уровню образования. Для этого выделим словарь и удалим из него дубликаты.

In [64]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21452 entries, 0 to 21451
Data columns (total 15 columns):
children              21452 non-null int64
days_employed         21452 non-null int64
dob_years             21452 non-null int64
education             21452 non-null object
education_id          21452 non-null int64
family_status         21452 non-null object
family_status_id      21452 non-null int64
gender                21452 non-null object
income_type           21452 non-null object
debt                  21452 non-null int64
total_income          21452 non-null int64
purpose               21452 non-null object
lemmas                21452 non-null object
purpose_group         21452 non-null object
total_income_group    21452 non-null object
dtypes: int64(7), object(8)
memory usage: 2.5+ MB


In [65]:
education_dict = df[['education_id', 'education']]
education_dict.head()

,education_id,education
0,0,высшее
1,1,среднее
2,1,среднее
3,1,среднее
4,1,среднее


In [66]:
education_dict = education_dict.drop_duplicates().reset_index(drop=True)
education_dict.head()

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


#### Категоризация по статусу семейного положения 

Проведем категоризацию по семейному статусу: выделим словарь и удалим из него дубликаты.

In [67]:
family_status_dict = df[['family_status_id', 'family_status']]
family_status_dict.head()

,family_status_id,family_status
0,0,женат / замужем
1,0,женат / замужем
2,0,женат / замужем
3,0,женат / замужем
4,1,гражданский брак


In [68]:
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True)
family_status_dict.head()

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


Удалим из таблицы df столбцы *education* и *family_status* для оптимизации скорости запросов.

In [69]:
df.drop(['education', 'family_status'], axis='columns', inplace=True)
df.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,lemmas,purpose_group,total_income_group
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",недвижимость,4
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",автомобиль,2
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",недвижимость,3
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",образование,4
4,0,11739,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба,3


### Вывод

Провели категоризацию данных по целям получения кредита, ежемесячному доходу, <font color='purple'>образованию и семейному статусу</font>. Используем полученные категории для ответов на вопросы.

### Шаг 3. Анализ влияния параметров заемщика на возврат кредита в срок

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Вычислим процент должников в зависимости от наличия детей.

In [70]:
children_absent = df[df['children'] == 0]
children_exist = df[df['children'] > 0]
children_absent_debt = children_absent[children_absent['debt']== 1].count()
children_exist_debt = children_exist[children_exist['debt']== 1].count()
children_absent_total = children_absent['children'].count()
children_exist_total = children_exist['children'].count()

In [71]:
children_absent_ratio = children_absent_debt / children_absent_total * 100
children_absent_ratio

children              7.544358
days_employed         7.544358
dob_years             7.544358
education_id          7.544358
family_status_id      7.544358
gender                7.544358
income_type           7.544358
debt                  7.544358
total_income          7.544358
purpose               7.544358
lemmas                7.544358
purpose_group         7.544358
total_income_group    7.544358
dtype: float64

In [72]:
children_exist_ratio = children_exist_debt / children_exist_total * 100
children_exist_ratio

children              9.195871
days_employed         9.195871
dob_years             9.195871
education_id          9.195871
family_status_id      9.195871
gender                9.195871
income_type           9.195871
debt                  9.195871
total_income          9.195871
purpose               9.195871
lemmas                9.195871
purpose_group         9.195871
total_income_group    9.195871
dtype: float64

Рассмотрим подробнее, как влияет количество детей на возврат кредита в срок.

In [73]:
def children_ratio(children):
    children_debt = df[(df['children'] == children) & (df['debt']== 1)].loc[:, 'children'].count()
    children_total = df[df['children'] == children].loc[:, 'children'].count()
    children_ratio = children_debt / children_total * 100
    return '{:.2f}'.format(children_ratio)

In [74]:
data = [[children_ratio(0), 
        children_ratio(1), 
        children_ratio(2), 
        children_ratio(3), 
        children_ratio(4), 
        children_ratio(5)]]
columns = ['0','1','2','3', '4', '5']
table = pd.DataFrame(data = data, columns = columns)
table

,0,1,2,3,4,5
0,7.54,9.15,9.49,8.18,9.76,0.00


### Вывод

Наличие детей отрицательно сказывается на возврате кредита в срок: процент должников без детей - 7,54%, с детьми - 9,2%. При этом наименьший процент должников среди клиентов с детьми - у клиентов с 3 детьми (для клиентов с 5 детьми нет данных о должниках). Такое влияние наличия детей на возврат кредита можно объяснить постоянно растущей степенью материального обеспечения детей. При сложным материальных положениях клиент предпочитает не вернуть долг по кредиту, но обеспечить детей. То, что клиенты с 3 детьми чаще возвращают кредит в срок, можно объяснить поддержкой государства многодетных семей. Льготы и пособия от государства получают также семьи с 4 детьми, но количество их не существенно повышено, по сравнению с семьями с 3 детьми.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Вычислим, как влияет семейное положение на возврат кредита в срок. Для этого рассчитаем процент должников для каждого статуса.

Использовала family_status_dict

In [77]:
family_status = family_status_dict
family_status

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


In [78]:
def family_status_ratio(family_status_id):
    family_status_debt = df[(df['family_status_id'] == family_status_id) & (df['debt']== 1)].loc[:, 'family_status_id'].count()
    family_status_total = df[df['family_status_id'] == family_status_id].loc[:, 'family_status_id'].count()
    family_status_ratio = family_status_debt / family_status_total * 100
    return '{:.2f}'.format(family_status_ratio)

In [79]:
family_status['family_status_ratio'] = family_status['family_status_id'].apply(family_status_ratio)
family_status

,family_status_id,family_status,family_status_ratio
0,0,женат / замужем,7.54
1,1,гражданский брак,9.35
2,2,вдовец / вдова,6.57
3,3,в разводе,7.11
4,4,Не женат / не замужем,9.75


### Вывод

Человек, хоть раз состоящий официально в браке, является более стабильным для банка (статусы - женат/замужем, вдовец/вдова, в разводе). При этом наименьший процент задолженностей у клиентов в статусе "вдовец/вдова" - 6,57%. У клиентов в статусе "гражданский брак" и "не женат/не замужем" близкие значения процента должников - 9,35 и 9,75 соответственно. Возможно люди, состоявшие в браке, более ответственны, серьезнее относятся к обязательствам и долгу.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Используем категоризацию по ежемесячноу доходу для ответа на поставленный вопрос: рассмотрим долю должников для каждой категории.

In [80]:
income_group_table

,group_name,income_from,income_to
0,1,20666,107801
1,2,107801,142594
2,3,142594,195546
3,4,195546,2265605


In [81]:
def total_income_ratio(group_name):
    total_income_debt = df[(df['total_income_group'] == group_name) & (df['debt'] == 1)].loc[:, 'total_income'].count()
    total_income_total = df[df['total_income_group'] == group_name].loc[:, 'total_income'].count()
    total_income_ratio = total_income_debt / total_income_total * 100
    return '{:.2f}'.format(total_income_ratio)

In [82]:
total_income = [[total_income_ratio('1'), 
        total_income_ratio('2'), 
        total_income_ratio('3'), 
        total_income_ratio('4')]]
columns = ['1','2','3','4']
table = pd.DataFrame(data = total_income, columns = columns)
table

,1,2,3,4
0,7.92,8.77,8.62,7.17


### Вывод

Самый низкий процент должников среди клиентов, чей ежемесячный доход выше, чем у 75% клиентов - 7,17% должников. Это логично: чем выше доход, тем выше возможность оплатить кредит вовремя. На втором месте клиенты с наименьшим уровнем дохода - 7,92% должников. Это можно объяснить тем, что клиенты с низким уровнем дохода более ответственно относятся к операциям с деньгами и лучше умеют ими распоряжаться. У клиентов из групп 2 и 3 (доход выше, чем у 25% клиентов и ниже, чему у 75% клиентов) процент задолженности выше - 8,77% и 8,62% соответственно. 

- Как разные цели кредита влияют на его возврат в срок?

In [83]:
df_pivot_table = df.pivot_table(index=['purpose_group'], columns='debt', values='children', aggfunc='count').reset_index()
df_pivot_table

debt,purpose_group,0,1
0,автомобиль,3903,403
1,недвижимость,10028,781
2,образование,3643,370
3,свадьба,2138,186


In [84]:
df_pivot_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 3 columns):
purpose_group    4 non-null object
0                4 non-null int64
1                4 non-null int64
dtypes: int64(2), object(1)
memory usage: 224.0+ bytes


In [85]:
df_pivot_table.set_axis(['purpose_group', 'no_debt', 'debt'], axis = 'columns', inplace = True)
df_pivot_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 3 columns):
purpose_group    4 non-null object
no_debt          4 non-null int64
debt             4 non-null int64
dtypes: int64(2), object(1)
memory usage: 224.0+ bytes


In [86]:
df_pivot_table['ratio'] = df_pivot_table.loc[:, 'debt'] / (df_pivot_table.loc[:, 'no_debt'] + df_pivot_table.loc[:, 'debt']) * 100
df_pivot_table

,purpose_group,no_debt,debt,ratio
0,автомобиль,3903,403,9.359034
1,недвижимость,10028,781,7.225460
2,образование,3643,370,9.220035
3,свадьба,2138,186,8.003442


### Вывод

Чаще всего в срок возвращают кредиты по приобретению недвижимости. Процент невозвратов в срок всего 7,2%. На втором месте  - организация свадьбы - 8% должников. Наименее стабильными  должниками являются клиенты, получившие кредит на образование или покупку автомобиля: 9,2% и 9,36% невозвратов в срок соответственно. Возможно такое распределение связано с заинтересованностью в получении той или иной цели. Потребности человека в семье и жилье выше остальных, только после удовлетворения этих потребностей, человек задумывается об образовании и автомобиле, но потребность и соответственно заинтересованность в них ниже. 

### Шаг 4. Общий вывод

Наиболее стабильным для банка является клиент, состоящий в браке, без детей, с доходом выше 195546, взявший кредит на недвижимость. Это человек ответственный (состоит в браке), с высоким доходом, не обременен расходами на детей, скорее всего выплатит кредит вовремя. 

Наиболее велика вероятность того, что клиент, не состоящий в браке, с детьми, с средним доходом (107801	- 195546), взявший кредит на автомобиль, не вернет кредит в срок.  

В дальнейшем можно рассмотреть зависимость возврата кредита в срок от пола и возраста клиента. 

Вопросы для работы в команде: как появляются в таблице клиенты с возрастом 0, клиенты с отрицательным трудовым стажем, пенсионеры и безработные с увеличенным в примерно 30 раз трудовым стажем, пол XNA, возможность заполнения возраста и количества детей выбором из списка, чтобы исключить ошибки при вводе данных.